In [66]:
from fastapi import FastAPI, HTTPException
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import sqlite3



In [28]:
model = pickle.load(open('model/advertising_model.pkl','rb'))
model

Pipeline(steps=[('polynomial_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('linear_regression', LinearRegression())])

## Data reading and split for example

In [70]:
df_adv = pd.read_csv('./data/raw/Advertising.csv',index_col=0)

In [71]:
df_adv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


In [89]:
x1 = df_adv.drop(columns='sales')
y1 = df_adv['sales']
x1, y1 = shuffle(x1, y1, random_state=42)
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1, random_state=42)

In [32]:
pickle.dump(x1_train,open('./data/processed/train/x1_train.pkl','wb'))
pickle.dump(x1_test,open('./data/processed/test/x1_test.pkl','wb'))
pickle.dump(y1_train,open('./data/processed/train/y1_train.pkl','wb'))
pickle.dump(y1_test,open('./data/processed/test/y1_test.pkl','wb'))

In [33]:
y_pred = model.predict(x1_test)

In [42]:
model.predict([[100, 100, 200]])

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


array([6035.03543148])

In [41]:
x1_test.iloc[0,:]

TV           149.7
radio         35.6
newspaper      6.0
Name: 194, dtype: object

In [34]:
y_pred

array([17387.21643844, 12095.22699877, 11219.71089179, 24522.89619423,
       12189.20902883, 15826.83686804,  7891.15011213, 15169.92220233,
       14446.35534332,  7158.99543034, 21110.30279181,  9442.26737948,
       20112.26724128, 12033.02944692, 17856.64128987, 14602.82849605,
       10499.41546749, 17238.14549997, 10556.17463049, 10907.51162037,
       12302.55203929,  6405.26398024,  6667.01409007, 17385.61494841,
        6631.30270373, 14948.34827621,  9815.95280148,  6398.52956095,
       15440.9132941 , 11689.06527406, 15527.03963895, 11227.32047725,
       10992.23171465, 15978.5122267 , 19976.05946553, 19227.7517418 ,
        9301.52209278, 18766.4225996 , 13815.46999276, 15834.75368425,
       22049.19319663, 19594.95341721, 12132.57855169,  9257.08004006,
        4547.09338959,  6976.98957594,  5215.8032184 , 18761.03560932,
        9789.26303585, 13947.19532847])

In [44]:
tv = 100  #pred.tv
radio = 100 #pred.radio
news = 200 #pred.newspaper
model.predict([[tv,radio,news]])[0]

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


np.float64(6035.03543147912)

In [47]:
ingest = x1_test.head(3)

In [46]:
y1_test.head(3)

194    17300.0
177    11700.0
9      10600.0
Name: sales, dtype: float64

In [59]:
ingest_results = np.array(y1_test[:3])
ingest_results

array([17300., 11700., 10600.])

In [ ]:
ingest.to_csv('./data/ingest/ingest.csv')

In [72]:
conn = sqlite3.connect('data/database.db')
df_adv.to_sql('advertising', conn, if_exists='replace',index=False)
conn.close()

# Pruebas para reentrenar

In [109]:
conn = sqlite3.connect('data/database.db')
cursor = conn.cursor()

In [ ]:
cursor.execute(
        '''
        SELECT * FROM advertising;
        '''
    )
results = cursor.fetchall()

In [ ]:
results

230.1

In [ ]:
res_dict={'TV':[],'radio':[],'newspaper':[],'sales':[]}

for result in results:
    res_dict['TV'].append(result[0])
    res_dict['radio'].append(result[1])
    res_dict['newspaper'].append(result[2])
    res_dict['sales'].append(float(result[3]))
df_res = pd.DataFrame(res_dict)
x1_train = df_res.drop(columns='sales')
y1_train = df_res['sales']
model.fit(x1_train,y1_train)
pickle.dump(model,open('./model/advertising_model.pkl','wb'))

In [108]:
model.fit(x1_train,y1_train)

Pipeline(steps=[('polynomial_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('linear_regression', LinearRegression())])

In [110]:
data = {'data': [[100, 100, 200]]}

In [112]:
data['data'][0][0]

100

In [113]:
class NewData():
    tv: int
    radio: int
    newspaper: int
    sales: int

In [ ]:
data = NewData()